In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
def e_detector_sr(data, lmbda, alt, w):
    N = len(data)
    E = np.zeros(N)
    M = np.zeros(len(alt))
    L = np.zeros(len(alt))
    for i in range(N):
        L = np.exp(lmbda-alt) * (alt/lmbda)**data[i]
        M = L*(M+1)
        E[i] = np.average(M, weights = w)
    return(E)

data = np.concatenate((np.random.poisson(100, 2000),np.random.poisson(120, 10),np.random.poisson(100, 2000)))
#data = np.random.poisson(100, 20000)
v=np.array([0,1,2,3,4,5,6,7,8,9])
s=np.array([1,2,3,4,5,6,7,8,9,np.inf])
wt = np.exp(-v)-np.exp(-s)
plt.plot(e_detector_sr(data, 100, np.array([110,120,130,140,150,160,170,180,190,200]), wt))
plt.xlim((0,len(data)))
plt.show()

In [ ]:
def e_detector_cusum(data, lmbda, alt, w):
    N = len(data)
    E = np.zeros(N)
    M = np.zeros(len(alt))
    L = np.zeros(len(alt))
    for i in range(N):
        L = np.exp(lmbda-alt) * (alt/lmbda)**data[i]
        M = L*np.maximum(M,1)
        E[i] = np.average(M, weights = w)
    return(E)

data = np.concatenate((np.random.poisson(100, 2000),np.random.poisson(120, 10),np.random.poisson(100, 2000)))
#data = np.random.poisson(100, 200000)
v=np.array([0,1,2,3,4,5,6,7,8,9])
s=np.array([1,2,3,4,5,6,7,8,9,np.inf])
wt = np.exp(-v)-np.exp(-s)
plt.plot(e_detector_cusum(data, 100, np.array([110,120,130,140,150,160,170,180,190,200]),wt))
plt.xlim((0,len(data)))
plt.show()

In [ ]:
alpha_levels = np.array([100000,150000,200000])
λ = 36
alt = np.array([45,55,65,75,85,95,105,115,125,135])
runs = range(50)
size=2000000



data_esr1=pd.DataFrame(index=alpha_levels, columns=runs)
for run in runs:
    print(run)
    np.random.seed(run)
    signal = np.random.poisson(λ, size)
    significance = e_detector_sr(signal, λ, alt, wt)
    for alpha_level in alpha_levels:
        run_length = np.argmax(significance>alpha_level)
        if run_length == 0:
            run_length = size
        data_esr1.loc[alpha_level, run] = run_length

data_esr1["average"] = data_esr1.mean(axis="columns")

In [ ]:
plt.plot(alpha_levels,data_esr1["average"])
plt.show()

In [ ]:
print(data_esr1["average"])

In [ ]:
##Linearly interpolate to find the threshold corresponding to ARL 481039.72, which is the ARL of poisson-focus with 5 sigma threshold

print(100000+(481039.72-325594.58)/4)

In [ ]:
alpha_levels = np.linspace(10**4,10**5, 20)
λ = 36
alt = np.array([45,55,65,75,85,95,105,115,125,135])
runs = range(50)
size=2000000


data_ecs=pd.DataFrame(index=alpha_levels, columns=runs)
for run in runs:
    np.random.seed(run)
    signal = np.random.poisson(λ, size)
    significance = e_detector_cusum(signal, λ, alt, wt)
    for alpha_level in alpha_levels:
        run_length = np.argmax(significance>alpha_level)
        if run_length == 0:
            run_length = size
        data_ecs.loc[alpha_level, run] = run_length

data_ecs["average"] = data_ecs.mean(axis="columns")

In [ ]:
plt.plot(alpha_levels,data_ecs["average"])
plt.show()

In [ ]:
print(data_ecs["average"])

In [ ]:
##Linearly interpolate to find the threshold corresponding to ARL 481039.72, which is the ARL of poisson-focus with 5 sigma threshold

print(60005.263158+(481039.72-434783.88)*(114210.526316-62105.263158)/(694252.32-434783.88))